In [7]:

# Part I: MPC Sum using Additive Secret Sharing
import random

# Private values (salaries)
ahmed_value = 80
ali_value   = 60
rashida_value = 80

# Function to split a secret into 3 additive shares
def split_into_3_shares(secret):
    # Generate two random shares
    s1 = random.randint(-1000, 1000)
    s2 = random.randint(-1000, 1000)
    # Third share ensures s1 + s2 + s3 = secret
    s3 = secret - s1 - s2
    return [s1, s2, s3]

# Each party splits their secret
ahmed_shares = split_into_3_shares(ahmed_value)
ali_shares   = split_into_3_shares(ali_value)
rashida_shares = split_into_3_shares(rashida_value)

print(f"Ahmed's shares: {ahmed_shares}")
print(f"Ali's shares: {ali_shares}")
print(f"Rashida's shares: {rashida_shares}")

# Distribute shares: index 0 → Ahmed, 1 → Ali, 2 → Rashida
# Each person receives one share from each sender
ahmed_received = [ahmed_shares[0], ali_shares[0], rashida_shares[0]]
ali_received   = [ahmed_shares[1], ali_shares[1], rashida_shares[1]]
rashida_received = [ahmed_shares[2], ali_shares[2], rashida_shares[2]]

# Each sums their received shares
sum_ahmed = sum(ahmed_received)
sum_ali   = sum(ali_received)
sum_rashida = sum(rashida_received)

print(f"\nAhmed's local sum: {sum_ahmed}")
print(f"Ali's local sum: {sum_ali}")
print(f"Rashida's local sum: {sum_rashida}")

# Global sum = sum of all local sums
global_sum = sum_ahmed + sum_ali + sum_rashida
print(f"\n Reconstructed total sum: {global_sum}")
print(f"Expected sum: {ahmed_value + ali_value + rashida_value}")
print(f"Correct? {global_sum == 220}")


# Part II: Shamir’s Secret Sharing (k=3, n=5)
from sympy import Poly, GF, invert
from sympy.abc import x
import random

PRIME = 2**127 - 1  # Large prime for finite field

def generate_polynomial(secret, k):
    """Generate random polynomial of degree k-1 with f(0) = secret"""
    coeffs = [secret] + [random.randint(1, PRIME - 1) for _ in range(k - 1)]
    return Poly(coeffs, x, modulus=PRIME)

def create_shares(poly, n):
    """Evaluate polynomial at x=1 to x=n to create shares"""
    return [(i, int(poly.eval(i) % PRIME)) for i in range(1, n + 1)]

def reconstruct_secret(shares, k):
    """Lagrange interpolation to recover secret from k shares"""
    x_vals = [s[0] for s in shares[:k]]
    y_vals = [s[1] for s in shares[:k]]
    secret = 0
    for i in range(k):
        numerator = 1
        denominator = 1
        for j in range(k):
            if i != j:
                numerator = (numerator * (-x_vals[j])) % PRIME
                denominator = (denominator * (x_vals[i] - x_vals[j])) % PRIME
        lagrange_coeff = (numerator * invert(denominator, PRIME)) % PRIME
        secret = (secret + y_vals[i] * lagrange_coeff) % PRIME
    return secret

# Parameters
secret = 12345
k = 3  # threshold
n = 5  # total shares

# Generate polynomial and shares
poly = generate_polynomial(secret, k)
shares = create_shares(poly, n)

print(f"Original secret: {secret}")
print(f"Generated {n} shares (first 3 shown):")
for i in range(3):
    print(f"  Share {shares[i][0]}: {shares[i][1]}")

# Reconstruct from first k shares
recovered = reconstruct_secret(shares, k)
print(f"\nRecovered secret: {recovered}")
print(f" Match? {recovered == secret}")


# Part III: Garbled Circuit for AND Gate


import secrets

def random_label():
    return secrets.token_hex(16)  # 128-bit random label

# Generate random labels for inputs and output
A_labels = [random_label(), random_label()]  # A=0, A=1
B_labels = [random_label(), random_label()]  # B=0, B=1
O_labels = [random_label(), random_label()]  # Output=0, Output=1

# Truth table for AND: only (1,1) → 1; others → 0
garbled_table = [
    (A_labels[0], B_labels[0], O_labels[0]),  # 0 AND 0 = 0
    (A_labels[0], B_labels[1], O_labels[0]),  # 0 AND 1 = 0
    (A_labels[1], B_labels[0], O_labels[0]),  # 1 AND 0 = 0
    (A_labels[1], B_labels[1], O_labels[1]),  # 1 AND 1 = 1
]

# Simulate inputs: e.g., Omar chooses A=1, Nancy chooses B=1
input_A = 1
input_B = 1

label_A = A_labels[input_A]
label_B = B_labels[input_B]

# Evaluator (say, Omar) looks up matching row
output_label = None
for a_lbl, b_lbl, out_lbl in garbled_table:
    if a_lbl == label_A and b_lbl == label_B:
        output_label = out_lbl
        break

# Map output label back to bit (only evaluator knows mapping)
if output_label == O_labels[1]:
    output_bit = 1
else:
    output_bit = 0

print(f"Omar's input A: {input_A}")
print(f"Nancy's input B: {input_B}")
print(f"Garbled output: {output_bit}")
print(f" Expected AND result: {input_A & input_B}")
print(f"Correct? {output_bit == (input_A & input_B)}")

Ahmed's shares: [-65, 189, -44]
Ali's shares: [-332, -911, 1303]
Rashida's shares: [217, -467, 330]

Ahmed's local sum: -180
Ali's local sum: -1189
Rashida's local sum: 1589

 Reconstructed total sum: 220
Expected sum: 220
Correct? True
Original secret: 12345
Generated 5 shares (first 3 shown):
  Share 1: 60741633958532755697367196010131342930
  Share 2: 42190353150295089856400235595020009135
  Share 3: 23639072342057424015433275179908700030

Recovered secret: 79292914766770421538334156425242701415
 Match? False
Omar's input A: 1
Nancy's input B: 1
Garbled output: 1
 Expected AND result: 1
Correct? True
